In [ ]:
# Let's build our project

# Here we are building a mulitple input mixed data model that takes in the 3 inputs seperately. RBI, chaneel matrices remain 
# seperate but now instead of giving Spectrum Occupancy matrix as input we give only NSA and instead of Src Des matrix we 
# just give Src and Des node id. The NSA and Src_Des input are given in two different branches, in the next model they will 
# be given together on one branch. The output power are indices rather matrices. So its an Nx1 vector.

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

print(tf.config.list_physical_devices())

# number of iterations the matlab code ran or the number of times the network is simulated
itr = 50000
# Number of iterations the test data generation matlab code ran or the number of times the network is simulated
itr_test = 2000
# number of nodes in the network
N = 30
# We scale the non zero values of output power so that there is some sizable error if the model outputs a zero matrix
scaling_parameter = 1

# Reading the RBI data
df = pd.read_csv("Data for RBI TL 0.7 mean 50000_itr(N=30)/Data_rbi_vectors.csv")
#print(df)
df.shape

In [ ]:
# coverting it into numpy array
X_1 = df.to_numpy()
#print(tem)
X_1.shape

#Just for the 1332 dataset as the last array will be nans
#tem = tem[:-1]

In [ ]:
# a=list(range(0, itr))
# X_1 = np.zeros((itr,N))
# for i in a:
#     X_1[i:] = tem[i:]
#X_1

In [ ]:
# Reading the channel data
df1 = pd.read_csv("Data for RBI TL 0.7 mean 50000_itr(N=30)/Data_channel_matrices.csv")
#print(df1)
df1.shape

In [ ]:
# coverting it into numpy array
X_2 = df1.to_numpy()

In [ ]:
X_2 = X_2.reshape((itr,N,N))
#print(X_2)
X_2 = np.nan_to_num(X_2)
#X_2

In [ ]:
# Reading the Source_Destination_info data
df2 = pd.read_csv("Data for RBI TL 0.7 mean 50000_itr(N=30)/Data_Source_Destination_info_matrix(only Src Des).csv")
#print(df2)
df2.shape

In [ ]:
# coverting it into numpy array
X_3 = df2.to_numpy()
#print(X_3)
X_3.shape

In [ ]:
X_3 = X_3.astype(np.float64)
print(X_3.dtype)
#print(X_3)

In [ ]:
# Reading the Spectrum_occupancy data, i.e, NSA
df3 = pd.read_csv("Data for RBI TL 0.7 mean 50000_itr(N=30)/Data_spectrum_matrix.csv")
#print(df3)
df3.shape

In [ ]:
# coverting it into numpy array
X_4 = df3.to_numpy()
#print(X_4)
X_4.shape

# Its already in float64 so no need to convert

In [ ]:
# Reading the output power data
df4 = pd.read_csv("Data for RBI TL 0.7 mean 50000_itr(N=30)/Data_output1.csv")
#print(df4)
df4.shape

In [ ]:
# coverting it into numpy array
X_5 = df4.to_numpy()
#print(X_5)
print(X_5.shape)
X_5 = np.nan_to_num(X_5)

In [ ]:
# This Cell converts each of the (1,30) output of y_data to (1,6)

a=list(range(0, itr))
max1 = 0
count = 0
maxlen = 0
max_path = np.zeros(30)
max_path_ind = 0
X_5_new = np.zeros((itr,6))
for i in a:
    temp = X_5[i]
    maxlen = len(temp[np.nonzero(temp)])
    if maxlen > 6:
        X_5_new[i,:] = temp[0:6]
        temp_non_zero = temp[np.nonzero(temp)]
        X_5_new[i,5] = temp_non_zero[-1]
        continue
    X_5_new[i,:] = temp[0:6]
    #print("original")
    #print(X_5[i])
X_5_new.shape

In [ ]:
# Note: for 33569 itr data  the 150th and 7128 paths have length > 6 , ie, [ 6. 23. 27.  3. 15. 17. 18.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
#  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] and [30.  7.  8. 16.  3. 25. 12. 21.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
#  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

In [ ]:
y_data = X_5_new.astype(np.float64)
print(y_data.dtype)
#print(y_data_temp)
y_data.shape

# No need to unravel it as its already a vector

In [ ]:
X_1.dtype, X_2.dtype, X_3.dtype, X_4.dtype, y_data.dtype

In [ ]:
X_1.shape, X_2.shape, X_3.shape, X_4.shape, y_data.shape

In [ ]:
# scaling parameter for y_data so model gets greater penalty for error and also since output is sparse

y_data = y_data*scaling_parameter

In [ ]:
# converting y_data to tensor

y_data = tf.constant(y_data)
print(y_data.shape)
#y_data = tf.reshape(y_data, [itr,1,1,N*N])
#print(y_data.shape)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.core import Activation
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import concatenate
#from tensorflow.python.keras.optimizers import adam

In [ ]:
# Creating branches of the model

# Activations
a1 = 'relu'
a2 = None

# Branch 1, takes RBI vectors as input and processws it

rbi_input = Input(shape=(N,))
x1 = tf.keras.layers.BatchNormalization()(rbi_input)
#x1 = Dropout(0.3)(x1)
x1 = Dense(50, activation=a1)(x1)
#x1 = Dropout(0.3)(x1)
x1 = Dense(50, activation=a1)(x1)
#x1 = Dense(1000, activation=a2)(x1)
#x1 = Dense(1000, activation=a2)(x1)
out_1 = Dense(N, activation=a1)(x1)

# Branch 2, takes channel matrix as input and processes it

chan_input = Input(shape=(N, N))
x2 = tf.keras.layers.BatchNormalization()(chan_input)
#x2 = Dropout(0.3)(x2)
x2 = Dense(80, activation=a1)(x2)
#x2 = Dropout(0.3)(x2)
#x2 = Flatten()(chan_input)
x2 = Dense(80, activation=a1)(x2)
#x2 = Dense(1000, activation =a2)(x2)
#x2 = Dense(1000, activation =a2)(x2)
x2 = Flatten()(x2)
out_2 = Dense(N*N, activation=a1)(x2)

# Branch 3, takes Source Destination info and processes it

S_D_input = Input(shape=(2, ))
#x3 = Flatten()(S_D_input)
x3 = tf.keras.layers.BatchNormalization()(S_D_input)
#x3 = Dropout(0.3)(x3)
x3 = Dense(10, activation=a1)(x3)
#x3 = Dropout(0.2)(x3)
#x3 = Dropout(0.3)(x3)
x3 = Dense(10, activation=a1)(x3)
#x3 = Dropout(0.2)(x3)
#x3 = Dense(1000, activation =a2)(x3)
#x3 = Dense(1000, activation =a2)(x3)
#x3 = Flatten()(x3)
out_3 = Dense(10, activation=a1)(x3)

# Branch 4, takes Spectrum occupancy info and processes it

Spec_input = Input(shape=(1, ))
#x4 = Flatten()(Spec_input)
x4 = tf.keras.layers.BatchNormalization()(Spec_input)
#x4 = Dropout(0.3)(x4)
x4 = Dense(20, activation=a1)(x4)
#x4 = Dropout(0.3)(x4)
x4 = Dense(20, activation=a1)(x4)
#x4 = Dense(1000, activation =a2)(x4)
#x4 = Dense(1000, activation =a2)(x4)
#x4 = Flatten()(x4)
out_4 = Dense(10, activation=a1)(x4)

# Now we concatenate all the branches 

concatenated = concatenate([out_1, out_2, out_3, out_4]) ## concatenate the four branches
#x5 = Dropout(0.1)(cocatenated)
x5 = Dense(100, activation=a1)(concatenated)
#x5 = Dropout(0.3)(x5)
x5 = Dense(100, activation=a1)(x5)
#x5 = Dropout(0.3)(x5)
x5 = Dense(100, activation=a1)(x5)
#x5 = Dense(2000, activation=a2)(x5)
#x5 = Dense(2000, activation=a2)(x5)
#x5 = Dense(2000, activation=a2)(x5)
#x5 = Dropout(0.2)(x5)
out = Dense(6, activation=a1)(x5)
model = Model([rbi_input, chan_input, S_D_input, Spec_input], out, name='Model_3')

# Set optimizer

#opt = Adam(lr=0.01, decay=1e-6)
#sgd = SGD(lr=0.0, momentum=0.9)

# Compile the model

#model.compile(loss='mse',
#              optimizer = 'adam',
#              metrics=['accuracy'])

model.compile(loss=tf.keras.losses.mse, # mae is short for mean absolute error
              optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001), # SGD is short for stocastic gradient descent
              metrics=["accuracy"])

#model.compile(loss=tf.keras.losses.mse,
#              optimizer=tf.keras.optimizers.Adam(),
#              metrics=["accuracy"])

# Fit the model

# We also measure the time taken to compute
t = time.perf_counter()
history = model.fit([X_1, X_2, X_3, X_4], y_data, batch_size=32, epochs=100, validation_split = 0.03)
t_c = time.perf_counter() - t

In [ ]:
model.summary()

In [ ]:
# Now we read the test data to evaluate our model


# Reading the Test RBI data
df_test = pd.read_csv("Test Data for RBI 0.7 mean 2000_itr(N=30)/Test_Data_rbi_vectors.csv")
#print(df_test)
df_test.shape

In [ ]:
# coverting it into numpy array
tem_test = df_test.to_numpy()
#print(tem_test)
tem_test.shape
X_1_test = tem_test

In [ ]:
# Reading the Test channel data
df1_test = pd.read_csv("Test Data for RBI 0.7 mean 2000_itr(N=30)/Test_Data_channel_matrices.csv")
#print(df1_test)
df1_test.shape

In [ ]:
# coverting it into numpy array
X_2_test = df1_test.to_numpy()
X_2_test.shape

In [ ]:
X_2_test = X_2_test.reshape((itr_test,N,N))
#print(X_2_test)
X_2_test = np.nan_to_num(X_2_test)
#X_2_test

In [ ]:
# Reading the Test Source_Destination_info data

df2_test = pd.read_csv("Test Data for RBI 0.7 mean 2000_itr(N=30)/Test_Data_Source_Destination_info_matrix(only Src Des).csv")
#print(df2_test)
df2_test.shape

In [ ]:
# coverting it into numpy array
X_3_test = df2_test.to_numpy()
#print(X_3_test)
X_3_test.shape

In [ ]:
X_3_test = X_3_test.astype(np.float64)
print(X_3_test.dtype)
#print(X_3_test)

In [ ]:
# Reading the Test Spectrum_occupancy data, i.e, NSA

df3_test = pd.read_csv("Test Data for RBI 0.7 mean 2000_itr(N=30)/Test_Data_spectrum_matrix.csv")
#print(df3_test)
df3_test.shape

In [ ]:
# coverting it into numpy array
X_4_test = df3_test.to_numpy()
#print(X_4_test)
X_4_test.shape

In [ ]:
X_4_test = X_4_test.astype(np.float64)
print(X_4_test.dtype)
#print(X_4_test)

In [ ]:
# Reading the Test output power data

df4_test = pd.read_csv("Test Data for RBI 0.7 mean 2000_itr(N=30)/Test_Data_output1.csv")
#print(df4_test)
df4_test.shape

In [ ]:
# coverting it into numpy array
X_5_test = df4_test.to_numpy()
#print(X_5_test)
print(X_5_test.shape)
X_5_test = np.nan_to_num(X_5_test)

In [ ]:
# This Cell converts each of the (1,30) y_data_test output to (1,6)

a=list(range(0, itr_test))
max1 = 0
count = 0
maxlen = 0
max_path = np.zeros(30)
max_path_ind = 0
X_5_test_new = np.zeros((itr_test,6))
for i in a:
    temp = X_5_test[i]
    maxlen = len(temp[np.nonzero(temp)])
    if maxlen > 6:
        X_5_test_new[i,:] = temp[0:6]
        temp_non_zero = temp[np.nonzero(temp)]
        X_5_test_new[i,5] = temp_non_zero[-1]
        continue
    X_5_test_new[i,:] = temp[0:6]
    #print("original")
    #print(X_5[i])
X_5_test_new.shape

In [ ]:
y_data_test = X_5_test_new.astype(np.float64)
print(y_data_test.dtype)
#print(y_data_test)
y_data_test.shape

In [ ]:
X_1_test.dtype, X_2_test.dtype, X_3_test.dtype, X_4_test.dtype, y_data_test.dtype

In [ ]:
X_1_test.shape, X_2_test.shape, X_3_test.shape, X_4_test.shape, y_data_test.shape

In [ ]:
X_data_test = [X_1_test, X_2_test, X_3_test, X_4_test]

In [ ]:
# converting y_data_test to tensor

y_data_test = tf.constant(y_data_test)
print(y_data_test.shape)
#y_data_test = tf.reshape(y_data_test, [itr_test,1,1,N*N])
#print(y_data_test.shape)
y_data_test[0]

In [ ]:
# We scale the non zero values of Test output power so that there is some sizable error if the model outputs a zero matrix
#scaling_parameter = 100

y_data_test = y_data_test*scaling_parameter
y_data_test[0]

In [ ]:
# Now we make predictions using our model
import time
t = time.perf_counter()
y_pred = model.predict(X_data_test)
elapsed = time.perf_counter() - t
print(elapsed)
print(y_pred.shape)
print("For 1 prediction")
print(elapsed/itr_test)
t_c = t_c + elapsed

In [ ]:
# Total time taken to train and predict one sample
print("Total time taken to train and predict all sample")
print(t_c)

In [ ]:
# Evaluate the model on the test

t1 = time.perf_counter()
model.evaluate(X_data_test, y_data_test)
elapsed1 = time.perf_counter() - t1
print(elapsed1)

In [ ]:
x_axis = np.arange(1,(6)+1)
x_axis.shape
ind = 3
y_pred = np.round(y_pred)
print(y_pred[ind]/scaling_parameter)
y_data_test[ind]/scaling_parameter

In [ ]:
a = list(range(0, itr_test))
#for i in a:
#    plt.figure()
#    plt.plot(x_axis,y_data_test[i]/scaling_parameter,'b',x_axis,y_pred[i]/scaling_parameter,'r')
#plt.show()

In [ ]:
# summarize history for accuracy
plt.ylim((0.2, 1))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
#plt.ylim((20, 40))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save('saved_model/RBI_TL_SepTr_0.7_mean(30-11-2023)')
